In [ ]:
pip install torch torchvision torchaudio


In [ ]:
pip install transformers requests beautifulsoup4 

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
# import requests
# from bs4 import BeautifulSoup
import re

Create the bert model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased')


In [8]:
tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt')
print(tokens[0])
result = model(tokens)
print(result)
result.logits
print(int(torch.argmax(result.logits))+1)


tensor([ 101, 1135, 1108, 1363, 1133, 1180, 2707, 1151, 1618,  119, 2038,  102])
SequenceClassifierOutput(loss=None, logits=tensor([[-0.0134,  0.0757]], grad_fn=<AddmmBackward>), hidden_states=None, attentions=None)
2


# step 1: Predefined CLass

In [9]:
import random

class Propaganda:
    NEGATIVE = "NEGATIVE"
    POSITIVE = "POSITIVE"

class Review:
    def __init__(self,sentence,SUBJprop):
        self.sentence = sentence
        self.SUBJprop = SUBJprop
        self.propaganda = self.get_propaganda()        

    def get_propaganda(self):
        if int(self.SUBJprop) <=3 :
            return Propaganda.NEGATIVE
        else:
            return Propaganda.POSITIVE


class ReviewContainer:
    def __init__(self,reviews):
        self.reviews = reviews

    def get_sentence(self):
        return [x.sentence for x  in self.reviews]

    def get_propaganda(self):
        return [x.propaganda for x in self.reviews]

    def evenly_distribute(self):
        negative = list(filter(lambda x: x.propaganda == Propaganda.NEGATIVE, self.reviews))
        positive = list(filter(lambda x: x.propaganda == Propaganda.POSITIVE, self.reviews))
        negative_shrunk = negative[:len(positive)]
        self.reviews = positive + negative_shrunk
        random.shuffle(self.reviews)


# step 2: Load Data

In [37]:
import pandas as pd
import numpy as np
import nltk
import string
from string import digits

import nltk


reviews = []
data  = pd.read_excel('Data/finalDataset.xlsx', engine='openpyxl')
df = pd.DataFrame(data.astype(str) , columns = ['Sentence','SUBJprop'])
# iterate elements of attribute "Sentence" and "SUBJprop" and push to the array "reviews"

for index, row in df.iterrows():
    sentence = row['Sentence']
    prop = row['SUBJprop']
    reviews.append(Review(sentence,prop))


print("Total Rows:")
print(len(reviews))
print("Total Positive:")
print(len(list(filter(lambda x: x.propaganda == Propaganda.POSITIVE, reviews))))
print("Total Negative:")
print(len(list(filter(lambda x: x.propaganda == Propaganda.NEGATIVE, reviews))))
#print(reviews[0].getSUBJprop)
# sentenceArray = [ x.sentence for x  in reviews] 

Total Rows:
16774
Total Positive:
3780
Total Negative:
12994


AttributeError: 'Review' object has no attribute 'getSUBJprop'

Define a function to use the bert model

In [15]:

def propaganda_score(review_sentence):
    tokens = tokenizer.encode(review_sentence, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

# step 2: Prep Data (split into train and test set)

In [13]:
from sklearn.model_selection import train_test_split

neg_prop = list(filter(lambda x: x.propaganda == Propaganda.NEGATIVE, reviews))
pos_prop = list(filter(lambda x: x.propaganda == Propaganda.POSITIVE, reviews))

########################################################################################
#split trainig and DevTest dataset
neg_train, neg_devtest  = train_test_split(neg_prop , train_size=0.7, random_state = 42 )
pos_train, pos_devtest = train_test_split(pos_prop , train_size=0.7, random_state = 42 )
########################################################################################
#prepare training dataset
train = neg_train + pos_train
random.shuffle(train)
########################################################################################
#prepare development and test dataset
neg_dev, neg_test = train_test_split(neg_devtest , train_size=0.5, random_state = 42 )
pos_dev, pos_test = train_test_split(pos_devtest , train_size=0.5, random_state = 42 )

dev = neg_dev + pos_dev
random.shuffle(dev)

test = neg_test + pos_test
random.shuffle(test)
########################################################################################
print("Total Train:")
print(len(train))
print("Positive:")
print(len(pos_train))
print("Negative:")
print(len(neg_train))

print("\nTotal Development:")
print(len(dev))
print("Positive:")
print(len(pos_dev))
print("Negative:")
print(len(neg_dev))

print("\nTotal Test:")
print(len(test))
print("Positive:")
print(len(pos_test))
print("Negative:")
print(len(neg_test))

Total Train:
11741
Positive:
2646
Negative:
9095

Total Development:
2516
Positive:
567
Negative:
1949

Total Test:
2517
Positive:
567
Negative:
1950


# step 3: Seperate the attribute, originally our array has text and score. we want them to be a seperate array


In [14]:
train_container = ReviewContainer(train)
train_container.evenly_distribute()

train_sentence = train_container.get_sentence()   
train_propaganda = train_container.get_propaganda() 

development_sentence = [x.sentence for x in dev]
development_propaganda = [x.propaganda for x in dev]

test_sentence = [x.sentence for x in test]
test_propaganda = [x.propaganda for x in test]


# print("Toalt rows after balance training dataset:")
# print(len(train_sentence))

# print("Positive Propaganda:")
# print(train_propaganda.count(Propaganda.POSITIVE))

# print("Negative Propaganda:")
# print(train_propaganda.count(Propaganda.NEGATIVE))

In [35]:
e = np.random.normal(size=100)  
e_dataframe = pd.DataFrame(train_sentence)      
e_dataframe.rename(index=str, columns={0:'sentence'})
#data= e_dataframe['sentence'].apply(lambda x: propaganda_score(x[:512]))
#print(data)
# propaganda_score(train_sentence[300])

,sentence
0,Solemnly professed:
1,Ford can’t remember where and when the alleged...
2,"Perhaps, I don’t know."
3,"The concerns, which the department says can be..."
4,"The length of their prison terms, in part, fue..."
...,...
5287,"-1.8% Employed 22,125 22,216 91 0.4% Employmen..."
5288,"Fillmore, Pierce, Buchanan?"
5289,"From there, Murillo continues writing for anot..."
5290,"So much for “I am woman, hear me roar.”"


In [16]:
# resultScore=[]
# for x in train_sentence:
#     resultScore.append(propaganda_score(x))
# print(resultScore[0])

Token indices sequence length is longer than the specified maximum sequence length for this model (7227 > 512). Running this sequence through the model will result in indexing errors


RuntimeError: The size of tensor a (7227) must match the size of tensor b (512) at non-singleton dimension 1

# Cross Validation

In [ ]:
# KFold cross validation
# Basic example (MORE INFO)

from sklearn.model_selection import KFold


kf=KFold(n_splits=3)
for train_index, test_index in kf.split(train_sentence):
    print(train_index, test_index)

In [ ]:

#StratifiedKFold is better when we have unbalanced data, it makes sure that in training there is sufficient for the smallest class
from sklearn.model_selection import StratifiedKFold 
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB


# KFold cross validation - on our dataset

def get_score(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    return model.score(X_test, y_test)

folds = StratifiedKFold(n_splits=2)

scores_logistic = []
scores_svm = []
scores_rf = []
scores_nb = []

# passong training dataset to Cross Validation
Sample_Array_sentence = train_sentence
Sample_Array_propaganda = train_propaganda


# Bag of words
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
Sample_Array_sentence_vectors = vectorizer.fit_transform(Sample_Array_sentence)


for train_index, test_index in folds.split(Sample_Array_sentence_vectors,Sample_Array_propaganda):
    X_train, X_test, y_train, y_test = Sample_Array_sentence_vectors[train_index], Sample_Array_sentence_vectors[test_index], \
                                       Sample_Array_propaganda[train_index], Sample_Array_propaganda[test_index]

    scores_logistic.append(get_score(LogisticRegression(solver='liblinear',multi_class='ovr'), X_train, X_test, y_train, y_test))  
    scores_svm.append(get_score(SVC(gamma='auto'), X_train, X_test, y_train, y_test))
    scores_rf.append(get_score(RandomForestClassifier(n_estimators=40), X_train, X_test, y_train, y_test))
    scores_nb.append(get_score(GaussianNB(), X_train.toarray(), X_test.toarray(), y_train, y_test))


print("Score of Logistic Regression")
print(scores_logistic)
print("Score of SVM")
print(scores_svm)
print("Score of RandomForest")
print(scores_rf)
print("Score of Naive Bayes")
print(scores_nb)

In [ ]:
# this is the same what we have done before but with the Sklearn Package
# from sklearn.model_selection import cross_val_score
# cross_val_score(LogisticRegression(solver='liblinear',multi_class='ovr'), Sample_Array_sentence_vectors, Sample_Array_propaganda,cv=10)
# cross_val_score(SVC(gamma='auto'), Sample_Array_sentence_vectors, Sample_Array_propaganda,cv=10)
# cross_val_score(RandomForestClassifier(n_estimators=40),Sample_Array_sentence_vectors, Sample_Array_propaganda,cv=10)

## step 4: Bag Of Words

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
train_x_vectors = vectorizer.fit_transform(train_sentence) # return a matrix of either 0 or 1 # train_x is our text
test_x_vectors = vectorizer.transform(test_sentence)
underthreshold_x_vectors = vectorizer.transform(UnderThreshold)

# Todo:
# transform development set to vectors
development_x_vectors = vectorizer.fit_transform(development_sentence)
print(train_sentence[0])
print(train_x_vectors[2517].toarray())
print(len(train_x_vectors.toarray()))

print(test_x_vectors[0].toarray())
print(len(test_x_vectors.toarray()))

print(development_x_vectors[0].toarray())
print(len(development_x_vectors.toarray()))

# step 5: Classification SVM

In [ ]:
from sklearn import svm

clf_svm = svm.SVC(kernel='rbf',C=1, probability=True)
clf_svm.fit(train_x_vectors, train_propaganda)
clf_svm.predict(test_x_vectors[1])


# step 5: Decision Tree 

In [ ]:
from sklearn.tree import DecisionTreeClassifier

clf_dec = DecisionTreeClassifier()
clf_dec.fit(train_x_vectors, train_propaganda)

clf_dec.predict(test_x_vectors[1])

# step 5: Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

clf_gnb = GaussianNB()
clf_gnb.fit(train_x_vectors.toarray() , train_propaganda)

clf_gnb.predict(test_x_vectors[1].toarray())

# step 5: Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

clf_log = LogisticRegression()
clf_log.fit(train_x_vectors, train_propaganda)

clf_log.predict(test_x_vectors[1])


# VIP : Predict_Proba using Threshold

In [ ]:
SVMProbabilityPropaganda=[]
UnderThreshold = [] 
UnderThresholdProp = []


for i in range(len(test_x_vectors.toarray())):
  # check the dpositive propaganas
  if(clf_svm.predict_proba(test_x_vectors[i])[0][1]) < 0.60:
    # check the negative propagandas
    if(clf_svm.predict_proba(test_x_vectors[i])[0][0]) < 0.60:
      UnderThreshold.append(test_sentence[i])
      UnderThresholdProp.append(test_propaganda[i])



print(len(UnderThreshold))


In [ ]:
# second model Logistic regression
SecondUnderThreshold = [] 
SecondUnderThresholdProp = []

for j in range(len(UnderThreshold)):
  # check the dpositive propaganas
  if(clf_log.predict_proba(underthreshold_x_vectors[j])[0][1]) < 0.60:
    # check the negative propagandas
    if(clf_log.predict_proba(underthreshold_x_vectors[j])[0][0]) < 0.60:
      SecondUnderThreshold.append(UnderThreshold[j])
      SecondUnderThresholdProp.append(UnderThresholdProp[j])

# bag of words
SecondUnderThreshold_x_vectors = vectorizer.transform(SecondUnderThreshold)


print(len(SecondUnderThreshold))
# print(clf_log.predict(underthreshold_x_vectors[0]))

In [ ]:
# to know how previous step works
# test=clf_log.predict_proba(test_x_vectors[0])
# print(test)
# print(test[0])
# print(test[0][0])

# step 6: Evaluation

In [ ]:
# Mean Accuracy
# For Support Vector Machine
print(clf_svm.score(underthreshold_x_vectors,UnderThresholdProp))
print(type(test_x_vectors))
print(type(UnderThreshold))
# For Decision Tree
# print(clf_dec.score(test_x_vectors,test_propaganda))
# # For Decision Naive Bayes
# print(clf_gnb.score(test_x_vectors.toarray(),test_propaganda))
# # For Logistic Regression
# print(clf_log.score(test_x_vectors,test_propaganda))

In [ ]:
# F1 Scores
from sklearn.metrics import f1_score

# For Support Vector Machine
print(f1_score(UnderThresholdProp, clf_svm.predict(underthreshold_x_vectors),average = None, 
labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))
# trash for negative and neutral labels

# # For Support Decision Tree
# print(f1_score(UnderThresholdProp,clf_dec.predict(UnderThreshold),average = None, 
# labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))

# # For Support Naive Bayes
# print(f1_score(UnderThresholdProp,clf_gnb.predict(UnderThreshold.toarray()),average = None, 
# labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))

# # For Logistic Regression
# print(f1_score(UnderThresholdProp,clf_log.predict(UnderThreshold),average = None, 
# labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))


In [ ]:
from sklearn.metrics import f1_score

# For Support Vector Machine
print(f1_score(test_propaganda, clf_svm.predict(test_x_vectors),average = None, 
labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))
# trash for negative and neutral labels

# # For Support Decision Tree
# print(f1_score(test_propaganda,clf_dec.predict(test_x_vectors),average = None, 
# labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))

# # For Support Naive Bayes
# print(f1_score(test_propaganda,clf_gnb.predict(test_x_vectors.toarray()),average = None, 
# labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))

# # For Logistic Regression
# print(f1_score(test_propaganda,clf_log.predict(test_x_vectors),average = None, 
# labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))

In [ ]:
# Precision
from sklearn.metrics import precision_score

# For Support Vector Machine
print(precision_score(UnderThresholdProp, clf_svm.predict(underthreshold_x_vectors),average = None, 
labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))
# trash for negative and neutral labels

In [ ]:
# Recall

from sklearn.metrics import recall_score

# For Support Vector Machine
print(recall_score(UnderThresholdProp, clf_svm.predict(underthreshold_x_vectors),average = None, 
labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))
# trash for negative and neutral labels

In [ ]:
import numpy as np
# check number of positive and negative classes in training set
print("Number of NEGATIVE in training set:")
print(train_propaganda.count(Propaganda.NEGATIVE))
print("Number of POSITIVE in training set:")
print(train_propaganda.count(Propaganda.POSITIVE))


# check number of positive and negative classes in test set
y = np.array(test_propaganda)
number_of_NEGATIVE = (y == "NEGATIVE").sum()
number_of_POSITIVE = (y == "POSITIVE").sum()
print("\nNumber of NEGATIVE in test set:")
print(number_of_NEGATIVE)
print("Number of POSITIVE in test set:")
print(number_of_POSITIVE)

# check number of positive and negative classes in development set

y = np.array(development_propaganda)
Dnumber_of_NEGATIVE = (y == "NEGATIVE").sum()
Dnumber_of_POSITIVE = (y == "POSITIVE").sum()
print("\nNumber of NEGATIVE in development set:")
print(Dnumber_of_NEGATIVE)
print("Number of POSITIVE in development set:")
print(Dnumber_of_POSITIVE)


# step 7: Tuning our model (with Grid Search)

In [ ]:
from sklearn.model_selection import GridSearchCV

# tuned = svm.SVC()
parameters = {'kernel':('linear','rbf'),'C':(1,4,8,16,32,64)}
svc = svm.SVC()
clf = GridSearchCV(svc,parameters,cv=10)
clf.fit(train_x_vectors,train_propaganda)
clf.cv_results_


In [ ]:
import pandas as pd

df = pd.DataFrame(clf.cv_results_)
df

In [ ]:
df[['param_C','param_kernel','mean_test_score']]

In [ ]:
clf.best_params_